## Graphs

One of the common types of databases that we can build Q&A systems for are graph databases. LangChain comes with a number of built-in chains and agents that are compatible with graph query language dialects like Cypher, SparQL, and others (e.g., Neo4j, MemGraph, Amazon Neptune, Kùzu, OntoText, Tigergraph). They enable use cases such as:

Generating queries that will be run based on natural language questions,
Creating chatbots that can answer questions based on database data,
Building custom dashboards based on insights a user wants to analyze,

### Langchain / NEo4J

https://python.langchain.com/docs/use_cases/graph/quickstart

https://python.langchain.com/docs/use_cases/graph/integrations/


## Neo4j

https://neo4j.com/download/?utm_source=Google&utm_medium=PaidSearch&utm_campaign=Evergreenutm_content=EMEA-Search-SEMCE-DSA-None-SEM-SEM-NonABM&utm_term=&utm_adgroup=DSA-use-cases&gad_source=1&gclid=Cj0KCQjwwYSwBhDcARIsAOyL0fgPqbVOh-rLQ03sIhDFaJRRcYdZcbjzso7NRkROcNOLD4wWZ-E6uYUaAtKhEALw


## sandbox

https://login.neo4j.com/u/login/identifier?state=hKFo2SBTWUN5bmwxb0Y5U01wN0liWTlzdERCU1dmV0VUMk94bKFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIDY0Z3lLLWN0eThPRHFQMlNVZExBdDRhMUQzNXN0U1I5o2NpZNkgRHhobWlGOFRDZXpuSTdYb2kwOFV5WVNjTEdabms0a2U
_wcB


In [11]:
import getpass
import os

from dotenv import dotenv_values

config = dotenv_values(".env")

In [12]:

os.environ["OPENAI_API_KEY"] = config.get("OPENAI_API_KEY")

In [15]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = config.get("NEO4J")

In [5]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

In [6]:
movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

In [16]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Movie {imdbRating: FLOAT, id: STRING, released: DATE, title: STRING},Person {name: STRING},Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)


https://python.langchain.com/docs/use_cases/graph/integrations/graph_cypher_qa

In [17]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: 'Casino'})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name
Full Context:
[{'actor.name': 'Sharon Stone'}, {'actor.name': 'Joe Pesci'}, {'actor.name': 'James Woods'}, {'actor.name': 'Robert De Niro'}]

> Finished chain.


{'query': 'What was the cast of the Casino?',
 'result': 'The cast of Casino included Sharon Stone, Joe Pesci, James Woods, and Robert De Niro.'}

In [18]:
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: "Casino"})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name
Full Context:
[{'actor.name': 'Sharon Stone'}, {'actor.name': 'Joe Pesci'}, {'actor.name': 'James Woods'}, {'actor.name': 'Robert De Niro'}]

> Finished chain.


{'query': 'What was the cast of the Casino?',
 'result': 'The cast of Casino included Sharon Stone, Joe Pesci, James Woods, and Robert De Niro.'}